# Neural Networks
## Lab session 3: TensorFlow eager

### Basic mechanics of TensorFlow

In [4]:
import tensorflow as tf
a = tf.random_uniform(shape=[3, 2])
b = a + 2.0
print(b)

tf.Tensor(
[[2.1477053 2.7965155]
 [2.6615286 2.8650625]
 [2.0061755 2.0236745]], shape=(3, 2), dtype=float32)


In [ ]:
sess = tf.Session()
with sess.as_default():
    print(sess.run(b))

In [5]:
with sess.as_default():
    c = sess.run(1.5*b)
print(b)

Tensor("add:0", shape=(), dtype=float32)


In [1]:
# IMPORTANT: restart the runtime before carrying on
import tensorflow as tf
tf.enable_eager_execution()

C:\Users\simon\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
# Try again
a = tf.random_uniform(shape=[3, 2])
b = a + 2.0
print(b)

In [9]:
# Auto-differencing
import tensorflow.contrib.eager as tfe

def f(x):
    return tf.sin(x ** 2.0)

f_grad = tfe.gradients_function(f, ['x'])

f_grad(tf.constant(3.0))

[<tf.Tensor: id=28, shape=(), dtype=float32, numpy=-5.4667816>]

In [10]:
ff = tfe.gradients_function(f_grad)

In [12]:
# Using variables and implicit gradients.
w = tfe.Variable(0.3)

def f():
    return tf.sin(w)

f_grad = tfe.implicit_gradients(f)

f_grad()

[(<tf.Tensor: id=57, shape=(), dtype=float32, numpy=0.9553365>,
  <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.3>)]

### Classification on the Iris dataset

In [22]:
from sklearn import datasets, model_selection
dataset = datasets.load_iris()
X = dataset['data']
y = dataset['target']
Xtrain, Xtest, ytrain, ytest = \
            model_selection.train_test_split(X, y, stratify=y)

In [23]:
class NN(tf.keras.Model):
    
    def __init__(self):
        super(NN, self).__init__()
        self.hidden = tf.layers.Dense(15, activation=tf.nn.tanh)
        self.out = tf.layers.Dense(3)
        
    def call(self, x):
        return self.out(self.hidden(x))
    
net = NN()

In [24]:
# Play with the model variables.
net.variables

[]

In [25]:
net(tf.constant(X[0:1]))

<tf.Tensor: id=117, shape=(1, 3), dtype=float64, numpy=array([[ 2.36962899,  0.55703203, -1.55184501]])>

In [27]:
len(net.variables)

4

In [30]:
# Define the loss function
def loss(x, y):
    ypred = net(x)
    l = tf.losses.sparse_softmax_cross_entropy(\
                                               labels=y, \
                                               logits=ypred\
                                              )
    return l

In [32]:
# Instatiate loss and optimizer
loss_grad = tfe.implicit_gradients(loss)
opt = tf.train.GradientDescentOptimizer(0.001)

In [33]:
# Training
Xtrain_t = tf.constant(Xtrain)
ytrain_t = tf.constant(ytrain)

for i in range(500):
    
    grads = loss_grad(Xtrain_t, ytrain_t)
    opt.apply_gradients(grads)

In [41]:
# Accuracy
import numpy as np
ypred = net(tf.constant(Xtest))
ypred = tf.argmax(ypred, axis=1)
tmp = tf.cast(tf.equal(ypred, tf.constant(ytest.astype(np.int64))), tf.float32)
tf.reduce_mean(tmp)

<tf.Tensor: id=45232, shape=(), dtype=float32, numpy=0.7894737>